#Cadenas

Las cadenas en LangChain funcionan para poder crear flujos de trabajo, donde unimos distintos "bloques" para poder crear un sistemas con LLMs mas complejo.

Por ejemplo, si quieres crear un sistema el cual conecte distintas entradas y salidas de LLMs entre si con las cadenas puedes gestionar que modelo genera que información, con que prompt y la salida de ese modelo si funciona como entrada a otro.

Hay bastantes cadenas que LangChain tiene por default que puedes usar como bloques para crear tu sistema. Pero tambien puedes crear tus propias cadenas al igual que descargar distintas cadenas que ha hecho la comunidad desde el LangChain-Hub

## Cadenas más usadas

Hay bastantes cadenas las cuales estan integradas dentro de LangChain, pero en este cuadeno estaremos viendo las mas cómunes que te serán muy útiles para poder desarrollar distintos sistemas.

- LLMChain
- SequentialChain
- Math/transformation

## LLMChain
LLMChain es una de las cadenas que más usaras. Lo que hace es unir dos elementos para que puedas interactuar con las LLMs de manera mas sencilla.

Una un modelo LLM (Puede ser LLama, OpenAI, Cohere etc.) y los templates de prompts vistos en el cuaderno intro.ipynb.

In [ ]:
from langchain import LLMChain, OpenAI, PromptTemplate

prompt = '''Eres un asistente virtual experto en {tema} y respondes
            con una lista de 3 conceptos clave sobre el mismo
            Solo enumeras los tres conceptos'''
template = PromptTemplate.from_template(prompt)

In [ ]:
llm = OpenAI(openai_api_key=API)
cadena_LLM = LLMChain(llm=llm, prompt=template)

La cadena es el elemento con el que vas a intecartuar ahora para poder hacer predicciones.

Lo que hace ahora esta cadena es tomar tu input (El tema), darle formato al prompt que usará y envia el prompt construido al modelo para su generación de texto

In [ ]:
cadena_LLM.predict(tema="ingenieria civil")

# SequentialChain
Para muchos casos de usos solo enviar un texto para ser procesado no es suficiente, por lo que se requiere de una secuencia de procesos que se ejecuten en orden. Para esto se puede utilizar las cadenas SimpleSequentialChain o SequentialChain que permiten encadenar varios procesos de manera secuencial.

Cuando quieres que la salida que genera el modelo funcione como entrada para otro, este es una gran manera de hacerlo.

En este ejemplo veremos SequentialChain, ya que nos brinda mas flexibilidad que SimpleSequentialChain, pues puede recibir multiples entradas y generar multiples salidas.

Armamos la primera cadena que será la misma que hicimos en el ejercicio anterior



In [ ]:
llm = OpenAI(openai_api_key=API)
prompt = '''Eres un asistente virtual experto en {tema} y respondes
            con una lista de 3 conceptos clave sobre el mismo
            Solo enumeras los tres conceptos'''
template = PromptTemplate.from_template(prompt)
cadena_lista = LLMChain(llm=llm, prompt=template, output_key="lista_conceptos")


Armamos una cadena la cual va a recibir la salida de la cadena cadena_LLM y lo procesa para generar otro texto

In [ ]:
prompt = '''Eres un asistente virtual que recibe una lista de conceptos
            de un area de conocimiento y
            debe devolver cual de esos conceptos es mejor aprender primero.
            Los conceptos son: {lista_conceptos}'''
template = PromptTemplate.from_template(prompt)
cadena_inicio = LLMChain(llm=llm, prompt=template, output_key="donde_iniciar")

In [ ]:
from langchain.chains import SequentialChain
cadenas = SequentialChain(chains=[cadena_lista, cadena_inicio],
                          input_variables=["tema"],
                          output_variables=["lista_conceptos", "donde_iniciar"],
                          verbose=True)

In [ ]:
cadenas({"tema": "fisica"})

In [ ]:
from langchain.chains import SimpleSequentialChain
cadena_simple = SimpleSequentialChain(chains=[cadena_lista, cadena_inicio], verbose=True)
cadena_simple.run("Inteligencia artificial")

# Otros ejemplos

In [ ]:
# MathChain
from langchain import LLMMathChain
cadena_mate = LLMMathChain(llm=llm, verbose=True)
cadena_mate.run("Cuanto es 432*12-32+32?")

In [ ]:
# TransformChain
from langchain.chains import TransformChain


def eliminar_brincos(input):
    """Elimina los brincos de línea de un texto."""
    texto = input["texto"]
    return {"texto_limpio": texto.replace("\n", " ")}


cadena_transformacion = TransformChain(input_variables=["texto"],
                                        output_variables=["texto_limpio"],
                                        transform=eliminar_brincos)

prompt = '''\nEste es un texto \ncon brincos de\n línea.\n\n'''


cadena_transformacion.run(prompt)